# A Benchmark for Biomedical Knowledge Graph based Similarity 

This notebook will guide you through the evaluation of semantic similarity measures. [GitHub](https://github.com/liseda-lab/kgsim-benchmark/tree/master) explains how to use the benchmark data sets for calculation and evaluation of semantic similarity measures.

## Table of Contents
1. [Importing libraries](#importing)
2. [Selecting datasets](#datasets)
3. [Correlation calculation](#correlation)
4. [Correlation plotting](#plot-correlation)
5. [Protein-protein interaction prediction](#ppi-prediction)

### 1. Importing libraries  <a class="anchor" id="importing"></a>
**In this notebook, [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org) as well as the Python libraries Pandas, NumPy, Scikit-learn, Matplotlib and SciPy are used.**

You should have [Anaconda](https://www.anaconda.com) installed in your computer, which already installs Python 3, Jupyter, Scikit-learn, Pandas, NumPy, Matplotlib and SciPy. 

If you have any issues, the library versions used in the development of this notebook can be found in the table below.

| Library     | Version  |
|:-----------:|:--------:|
| Anaconda    | 5.3.0    |
| Matplotlib  | 3.0.3    |
| Numpy       | 1.15.4   | 
| Pandas      | 0.23.4   | 
| Scikit-learn| 0.20.1   |
| Scipy       | 1.1.0    |

In [ ]:
import ppi_prediction as ppi
import evaluating as evlt
import matplotlib.pyplot as plt
from IPython import display

### 2. Selecting datasets  <a class="anchor" id="datasets"></a>

In this section, you should state the type of benchmark data sets you're dealing with (`dataset_type` = PPI, MF or GENE, case-insensitive) as well as the path for your `test_dataset` and `benchmark_dataset`.

The datasets for `benchmark_dataset` can be retrieved from [GitHub](https://github.com/liseda-lab/kgsim-benchmark/tree/master/Data%20Sets).

**Beware** that using benchmark and test datasets of different sizes will raise an error in the evaluation and that selecting the wrong dataset_type can lead to wrongly calculated results.

In [ ]:
dataset_type = 'PPI, MF or GENE'
test_dataset = 'path_to_test_measure.csv'
benchmark_dataset = 'path_to_benchmark_dataset.csv'

`test_dataset` should be the path for the dataset containing the test semantic similarity measures. `extract_new_ssm` extracts only the column containing said measure (default is `col_n` = -1). The default delimiter for these datasets is `;`, however it can be changed by adding the parameter `sep`. Additionally, if the dataset has no header, the parameter `head` = `None` should be added. An example of how to use these parameters is in a comment in the cell below.


`measures` is the name of the dictionary containing all the measures you want to evaluate, the key being the measure's name (that will feature in the plots and dataframes throughout this notebook) and the value being the calculated similarities for the pairs in the data set using that measure.

In [ ]:
#If your target semantic similarity measure is in the first column of a headerless data set, use:
#'measure name':evlt.extract_new_ssm(test_dataset, col_n = 0, head = None)

measures = {'measure_name':evlt.extract_new_ssm(test_dataset, sep = '', col_n = int)}

### 3. Correlation calculation  <a class="anchor" id="correlation"></a>
This module will calculate Pearson Correlation Coefficient between the test semantic similarity measures and the similarity proxies available for the type of dataset you're working with. This produces a dataset that enables the comparison between state of the art semantic similarity measures and the test semantic similarity measures.

In [ ]:
results = evlt.correlation_calculation(benchmark_dataset, measures, dataset_type)
display.display(results)

### 4. Correlation plotting  <a class="anchor" id="plot-correlation"></a>
In this section you can plot each test semantic similarity measure and the similarity proxies available for the type of dataset you're working with. Semantic similarity measures can be plotted against the following proxies, according to dataset type:

| Dataset type| Similarity proxy  |
|:-----------:|-------------------|
| PPI         | sim(seq)|
| MF          | sim(seq) and sim(MF) |
| GENE        | sim(PS)   | 

You can costumize your plots by changing the labels for the axis (`xlabel` and `ylabel`) and by adding `size` = `int` in any of the plotting functions. This argument will change the size of the plot points. The color of the plot points can also be changed with the argument `col`. Check [Matplotlib](https://matplotlib.org/3.1.0/gallery/color/named_colors.html) for more information on colors and the comments in the cell below for information on how to use the alternative parameters.

If you're working with a MF dataset, two side by side plots will be produced. Alternativley, you can run the two cells below for separate plots.

In [ ]:
selected_measure = "measure_name"

In [ ]:
if evlt.molecular_function_dataset(dataset_type):
    plot = evlt.plot_molecular_function(benchmark_dataset, measures[selected_measure])
    plot[0].set(xlabel = 'Sequence Similarity', ylabel=selected_measure)
    plot[1].set(xlabel = 'Molecular Function Similarity', ylabel=selected_measure)
else:
    plot = evlt.correlation_plot(benchmark_dataset, measures[selected_measure], dataset_type, selected_measure) 
    #plt.ylabel('your new ylabel here')
    #plt.xlabel('your new xlabel here')
    
#if you want to costumize any of your plots, e.g. with large pink dots, you should add these parameters to the plotting functions:
#plot = evlt.plot_molecular_function(benchmark_dataset, measures[selected_measure], size = 5, col = 'hotpink')
#plot = evlt.correlation_plot(benchmark_dataset, measures[selected_measure], dataset_type, size = 5, col = 'hotpink') 

In [ ]:
if evlt.molecular_function_dataset(dataset_type):
    plot_pfam = evlt.plotting_molecular_function(benchmark_dataset, measures[selected_measure], selected_measure)
    #plt.ylabel('your new ylabel here')
    #plt.xlabel('your new xlabel here')

In [ ]:
if evlt.molecular_function_dataset(dataset_type):    
    plot_seq = evlt.plotting_sequence(benchmark_dataset, measures[selected_measure], selected_measure)
    #plt.ylabel('your new ylabel here')
    #plt.xlabel('your new xlabel here')

### 5. Protein-protein interaction  <a class="anchor" id="ppi-prediction"></a>
In case you're dealing with a PPI dataset, it's possible to calculate how well the semantic similarity measures can predict Protein Protein Interactions. This module has two different functionalities:
1. Plot a Precision-Recall plot and find the highest F1-score. This is found by using 10-fold cross validation to test a range of different thresholds, selecting the best one, for each measure (state of the art and test).  
2. Calculate precision, recall and F1-score for a given threshold (by changing the value of `thresh`). A dataframe and plot comparing precision, recall and F1-score for all semantic similarity measures at that threshold will be produced. The plot points color can be changed by adding the parameter `col`= [list of strings with size = number of measures], where each string is a valid color. You can also add `size` = `int` if you want to change the size of the plot points. 

For both plots, you can check [Matplotlib](https://matplotlib.org/3.1.0/gallery/color/named_colors.html) for more information on the colors for your plot, or check the comments for code examples. **Beware** that for both plots, the default color list only supports 10 different similarity measures. If you want to plot more than 10 measures in the same plot, you must input your own `col` list with Matplotlib colors.

In [ ]:
if ppi.interaction_dataset(dataset_type):
        results = ppi.plot_precision_recall(benchmark_dataset, measures)
        display.display(results)
    #if you don't want to calculate the best threshold for the benchmark measures add the parameter benchmark_measures = False

In [ ]:
#input your threshold of choice
thresh = 0.2 

if ppi.interaction_dataset(dataset_type):
        df_f1 = ppi.get_f1_score_df(benchmark_dataset, measures, thresh)
        ppi.plot_f1(df_f1)
        display.display(df_f1)

#### Authors

* **Carlota Cardoso** 
* **Rita Sousa**
* **Cátia Pesquita** 

#### License
See the [LICENSE.md](https://github.com/liseda-lab/kgsim-benchmark/blob/master/LICENSE.md/LICENSE.md) file for details.

#### Acknowledgments

This project was funded by the Portuguese FCT through the LASIGE Research Unit (UID/CEC/00408/2019), and also by the SMILAX project (PTDC/EEI-ESS/4633/2014).